# IBM Capstone Week 4
## Battle of the Neighborhoods
### Calvin Todorovich 6/4/20

##### Presentation is important

### Introduction

I decided to use Toronto, Canada as my city of choice for this project, since I checked every other postal code in Canada (A-Z), and more than 20 U.S. cities and none of them had any sort of table data similar to week 3's assignment. It honestly blows my mind that the page we were taught to scrape is utterly unique compared to every other postal code starting letter in Canada, and it has to be one of the most suspicious things I've ever encountered in my life. 

If someone were to open a restaurant, proximity to competing restuarants, number of potential customers (population), and parking could be good predictors for how well a restaurant will do.

In [2]:
#Setting up Libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### Data:

From foursquare, we will create a Venues data frame for Portland, OR. This dataframe will have Latitude, Longitude, Neighborhood, Number of Residents in Neighborhood, Venue Name, Venue Category as its columns, so we can examine how much competition and how many customers a new business should expect. 

In [3]:
#Toronto Data from last week

wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

class_id = 'wikitable sortable jquery-tablesorter'

response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, "html.parser")

canada_table = soup.find("table",{"class": "wikitable sortable"})

#I'm not sure if I parsed it wrong, but the canada_table saved as a list, which still had all the html tags in it
#So I found some user defined functions to translate html to csv

table = canada_table

def get_table_headers(table):
    headers = []
    for th in table.find("tr").find_all("th"):
        headers.append(th.text.strip())


df = pd.read_csv("can_table.csv")

#drop that extra unnamed row
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.Neighborhood.fillna(df.Borough, inplace = True)

#If a neighborhood is still unassigned, drop it
df = df.replace('Not assigned', np.nan).dropna()

df2 = pd.read_csv(r'C:\Users\Todo\Documents\Geospatial_Coordinates.csv')

TorLoc = pd.merge(left = df, right = df2)
TorLoc.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [5]:
#Set up Lat and Long
address = 'Toronto'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorLoc['Latitude'], TorLoc['Longitude'], TorLoc['Borough'], TorLoc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [9]:
#Setting up foursquare data

# define Foursquare Credentials and Version
CLIENT_ID = 'ZKXNTW0JK4NFBVDHSQMDD1KSQGZMSMG5WLZQSZQUPX0O04TT'
CLIENT_SECRET = '1U5YA0JRWGIRWM4P1AGQVEQWTDFKSWIULJT1VO2YIAFJILER'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZKXNTW0JK4NFBVDHSQMDD1KSQGZMSMG5WLZQSZQUPX0O04TT
CLIENT_SECRET:1U5YA0JRWGIRWM4P1AGQVEQWTDFKSWIULJT1VO2YIAFJILER


In [10]:
#Get Venues for Toronto
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(TorLoc['Latitude'], TorLoc['Longitude'], TorLoc['Postal Code'], TorLoc['Borough'], TorLoc['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [11]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2132, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M3A,North York,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,North York,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,M3A,North York,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
3,M3A,North York,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,M4A,North York,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


## Results:




### Discussion 
#### (of interesting observations and recommendations)

